In [1]:
import librosa
import librosa.display
import pretty_midi
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import os

versions = ['HU33', 'SC06', 'FI66', 'FI80', 'midi']

chroma_type = ['stft', 'cq', 'cens']
# Naming mistake, cq should be cqt


# I'll just choose the easy way out...
key2num = {
    "A:maj": 0, "Bb:maj": 1, "B:maj": 2,
    "C:maj": 3, "Db:maj": 4, "D:maj": 5, "Eb:maj": 6, 
    "E:maj": 7, "F:maj": 8, "F#:maj": 9, "G:maj": 10, "G#:maj": 11,
    "A:min": 12, "Bb:min": 13, "B:min": 14,
    "C:min": 15, "Db:min": 16, "D:min": 17, "D#:min": 18, "Eb:min": 18, 
    # There is D#:min and Eb:min, This could be a problem when finding key
    "E:min": 19, "F:min": 20, "F#:min": 21, "G:min": 22, "G#:min": 23,
    }

binary_templates = [[1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1], [1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0]]

ks_templates = [[6.35, 2.23, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.88], 
                [6.33, 2.68, 3.52, 5.38, 2.6, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17]]

# Tested with an irrelevent vector, perform a differnt result than binary_templates
#ks_templates = [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]

alpha = 0.9

uc = [1 + alpha + alpha**3 + alpha**7,
        0,
        0,
        0,
        alpha**4,
        0,
        0,
        alpha**2+alpha**5,
        0,
        0,
        alpha**6,
        0]


u = [None]*12
for i in range(12):
    # 0 means ua, 1 means ua#, ...
    u[i] = np.roll(uc, i-3)

harmonic_major_template = [0] * 12
harmonic_minor_template = [0] * 12

for i in range(12):
    if binary_templates[0][i] == 1:
        harmonic_major_template += u[(i+3)%12]
    if binary_templates[1][i] == 1:
        harmonic_minor_template += u[(i+3)%12]    

harmonic_templates = [harmonic_major_template, harmonic_minor_template]


In [2]:
def get_SWD_keys(versions, key2num):
    """
    Get keys for each song in each genres in GTZAN dataset. The path of the set is fixed.
    
    :param: versions
    :return: A dict that contains the true key of each genres.
    
    Example:
        key0 = d[1]['HU33']
    """
    
    d = {}    
    # Load audio key
    key_path = "data/SWD/02_Annotations/ann_audio_globalkey.csv"
    with open(key_path, mode='r') as f:
        
        # Skip first line
        line = f.readline()
        
        while True:
            
            # Parse the string
            line = f.readline()
            if not line:
                break
                
            line = line.split('\"')
            index = int(line[1].split('-')[1])
            
            version = line[3]
            if not version in versions:
                continue
                
            if not version in d:
                d[version] = {}
            
            if not line[5] in key2num:
                print('Error')
            
            d[version][index] = key2num[line[5]]
        # While there are two audio key files, the second file does not seem to be necessary to load.
        
    # Load score key
    key_path = "data/SWD/02_Annotations/ann_score_globalkey.csv"
    with open(key_path, mode='r') as f:
        # Parse the string
        
        # Skip first line
        line = f.readline()
        
        while True:
            
            line = f.readline()
            if not line:
                break

            line = line.split('\"')
            index = int(line[1].split('-')[1])
            
            version = 'midi'
                
            if not version in d:
                d[version] = {}
                
            if not line[3] in key2num:
                print('Error')
            
            d[version][index] = key2num[line[3]]
        
    
    for v in versions:
        d[v] = list(d[v].values())
        d[v].insert(0, -1)
    
    return d
    
    

In [3]:
keys = get_SWD_keys(versions, key2num)

In [4]:
keys['FI66']

[-1,
 15,
 22,
 17,
 12,
 7,
 17,
 17,
 20,
 12,
 14,
 10,
 14,
 3,
 14,
 14,
 5,
 5,
 17,
 0,
 20,
 8,
 21,
 10,
 22]

In [33]:
def create_SWD_Chroma(versions):
    """
    Create chroma file from mp3 and mid, save them to result/SWD/
    
    :param: versions(list): The list of versions. 
    :return: None
    """
    
    version2fileType = {"FI66": ".mp3", "FI80": ".mp3", "HU33": ".wav", "SC06": ".wav", "midi": ".mid"}
    

    if(not os.path.isdir('result/SWD/chroma')):
        for v in versions:
            
            # Skip midi for now
            if v == "midi":
                continue
            
            for i in range(24):
                # 24 is the numbers songs of each genres'data 

                filename = "data/SWD/01_RawData/audio_wav/" + v + "/Schubert_D911-" + str(i+1).zfill(2) + "_" + v + version2fileType[v]
                y, sr = librosa.load(filename)

                # STFT chromagram
                chroma_stft = librosa.feature.chroma_stft(
                    y=y, sr=sr, n_chroma=12, n_fft=4096)
                
                # CQT chromagram
                chroma_cq = librosa.feature.chroma_cqt(y=y, sr=sr)
                
                # Chroma Energy Normalized chromagram
                chroma_cens = librosa.feature.chroma_cens(y=y, sr=sr)
                
                # Create new files for three types of chroma
                
                filename = "result/SWD/chroma/" + v + "/stft/" + str(i+1).zfill(5) + ".chroma_stft.txt"
                os.makedirs(os.path.dirname(filename), exist_ok=True)
                with open(filename, mode='w') as f:
                    np.savetxt(filename, chroma_stft)
                    
                filename = "result/SWD/chroma/" + v + "/cq/" + str(i+1).zfill(5) + ".chroma_cq.txt"
                os.makedirs(os.path.dirname(filename), exist_ok=True)
                with open(filename, mode='w') as f:
                    np.savetxt(filename, chroma_cq)
                    
                filename = "result/SWD/chroma/" + v + "/cens/" + str(i+1).zfill(5) + ".chroma_cens.txt"
                os.makedirs(os.path.dirname(filename), exist_ok=True)
                with open(filename, mode='w') as f:
                    np.savetxt(filename, chroma_cens)

    v = "midi"
    for i in range(24):

        filename = "data/SWD/01_RawData/score_midi/Schubert_D911-" + str(i+1).zfill(2) + version2fileType[v]
        midi_data = pretty_midi.PrettyMIDI(filename)

        chroma_midi = midi_data.get_chroma(fs=100) # Not sure how fs supposed to be

        filename = "result/SWD/chroma/" + v + "/"  + str(i+1).zfill(5) + "." + v + ".txt"
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename, mode='w') as f:
            np.savetxt(filename, chroma_midi)
            
        
        
    return

In [34]:
create_SWD_Chroma(versions)

C:\Users\pride829\anaconda3\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


In [7]:
def get_SWD_Chroma(version:str, index:int, chroma_type:str=None):
    """
    Get chroma from txt
    
    :return: (ndarray)
    """
    
    if not version == "midi":
        filename = "result/SWD/chroma/" + version + "/" + chroma_type + "/" + str(index).zfill(5) + ".chroma_" + chroma_type + ".txt"
        with open(filename, mode='r') as f:
            return np.loadtxt(filename)
    else:
        filename = "result/SWD/chroma/" + version + "/"  + str(index).zfill(5) + "." + version + ".txt"
        with open(filename, mode='r') as f:
            return np.loadtxt(filename)

In [8]:
get_SWD_Chroma("FI66", 1, "cens")

array([[0.31588927, 0.31109607, 0.30639002, ..., 0.63326299, 0.63054359,
        0.62742054],
       [0.41401154, 0.4080548 , 0.40170783, ..., 0.08710974, 0.09396885,
        0.10104251],
       [0.42919198, 0.43215033, 0.43497959, ..., 0.22854227, 0.22922067,
        0.22984125],
       ...,
       [0.23344074, 0.23518088, 0.23710883, ..., 0.        , 0.        ,
        0.        ],
       [0.23285231, 0.23378283, 0.23464169, ..., 0.01722649, 0.01816434,
        0.01907932],
       [0.23285231, 0.23378283, 0.23464169, ..., 0.06988326, 0.0758045 ,
        0.08196318]])

In [9]:
get_SWD_Chroma("midi", 1)

array([[  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [ 65.,  65.,  65., ..., 126., 126., 126.],
       ...,
       [ 65.,  65.,  65., ...,  84.,  84.,  84.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]])

In [10]:
def create_SWD_tonic_pitch(versions:list, chroma_type:list):
    """
    Create tonic pitch list from chroma files if not already exsists.
    Start from A == 0
    
    """
    if(not os.path.isdir('result/SWD/tonic_pitch')):
        tonic_pitch_path = "result/SWD/tonic_pitch/"
        
        for v in versions:
            tonic_pitch_versions_path = tonic_pitch_path + v + "/"
            
            if v != "midi":
                for c in chroma_type:

                    filename = tonic_pitch_versions_path + c + ".txt"
                    os.makedirs(os.path.dirname(filename), exist_ok=True)
                    with open(filename, mode='w') as f:

                        for i in range(1, 25):
                        # 24 is the numbers of songs of each versions'data
                        # Data starts from 1
                            chroma = get_SWD_Chroma(v, i, c)
                            mean_pooling_chroma = chroma.sum(axis=1)
                            tonic_pitch = (np.argmax(mean_pooling_chroma)+3)%12 # Shift to A=0
                            f.write(str(tonic_pitch) + '\n')
            else:
                filename = tonic_pitch_versions_path + "midi.txt"
                os.makedirs(os.path.dirname(filename), exist_ok=True)
                with open(filename, mode='w') as f:

                    for i in range(1, 25):
                        chroma = get_SWD_Chroma(v, i)
                        mean_pooling_chroma = chroma.sum(axis=1)
                        tonic_pitch = (np.argmax(mean_pooling_chroma)+3)%12 # Shift to A=0
                        f.write(str(tonic_pitch) + '\n')
    return

In [11]:
create_SWD_tonic_pitch(versions, chroma_type)

In [12]:
def get_SWD_tonic_pitch(versions:list, chroma_type:list):
    """
    :Return: pitches(dict): 
    """
    
    d = {}
    tonic_pitch_path = "result/SWD/tonic_pitch/"
    
    for v in versions:
        if v != "midi":
            tonic_pitch_versions_path = tonic_pitch_path + v + "/"
            d[v] = {}

            for c in chroma_type:
                filename = tonic_pitch_versions_path + c + ".txt"

                with open(filename, mode='r+') as f:
                    d[v][c] = [-1] # No number 0
                    d[v][c].append(int(f.readline().rstrip()))
                    for i in range(2, 25):
                        d[v][c].append(int(f.readline().rstrip()))
        else:
            
            filename = tonic_pitch_path + "/midi/midi.txt"
            with open(filename, mode='r+') as f:
                d[v] = [-1]
                d[v].append(int(f.readline().rstrip()))
                
                for i in range(2, 25):
                    d[v].append(int(f.readline().rstrip()))
    
    return d

In [13]:
pitches = get_SWD_tonic_pitch(versions, chroma_type)
pitches['FI66']['cq']

[-1, 3, 5, 0, 7, 2, 0, 5, 3, 0, 2, 10, 2, 3, 2, 2, 5, 5, 5, 7, 8, 3, 4, 5, 10]

In [14]:
pitches['midi']

[-1, 0, 7, 3, 10, 2, 2, 2, 5, 2, 3, 0, 2, 1, 3, 3, 1, 5, 1, 7, 10, 3, 5, 7, 7]

In [15]:
def predict_SWD_key(chroma, major_template, minor_template, tonic_pitch: int):
    """
    Tonic_pitch start with A
    
    """
    
    shifted_major_template = np.roll(major_template, -3 + tonic_pitch)
    shifted_minor_template = np.roll(minor_template, -3 + tonic_pitch)
    # -3 becuase input tempalte tonic is C
    
    mean_pooling_chroma = chroma.sum(axis=1)
    
    major_pcc = stats.pearsonr(mean_pooling_chroma, shifted_major_template)[0]
    minor_pcc = stats.pearsonr(mean_pooling_chroma, shifted_minor_template)[0]
    
    if major_pcc > minor_pcc:
        return tonic_pitch # Major
    else:
        return tonic_pitch+12 # Minor
    

In [16]:
predict_SWD_key(get_SWD_Chroma('FI66', 1, 'cq'), 
                  binary_templates[0], 
                  binary_templates[1], 
                  pitches['FI66']['cq'][1])

15

In [17]:
predict_SWD_key(get_SWD_Chroma('midi', 1), 
                  binary_templates[0], 
                  binary_templates[1], 
                  pitches['midi'][1])

12

In [18]:
def create_SWD_predict(versions:list, chroma_type:list, pitches:dict, binary_templates:list, ks_templates:list, harmonic_templates:list):
    
    if(not os.path.isdir('result/SWD/predict/')):
        predict_path = "result/SWD/predict/"
        
        for v in versions:
            
            predict_versions_path = predict_path + "/" + v + "/"
            
            
            if v != "midi":
                for c in chroma_type:
                    predict_versions_chroma_path = predict_versions_path + "/" + c + "/"

                    # Get binary_templates results
                    filename = predict_versions_chroma_path + "binary_templates.txt"
                    os.makedirs(os.path.dirname(filename), exist_ok=True)
                    with open(filename, mode='w') as f:
                        for i in range(1, 25):
                            predict_key = predict_SWD_key(
                            get_SWD_Chroma(v, i, c),
                                binary_templates[0],
                                binary_templates[1],
                                pitches[v][c][i]
                            )
                            f.write(str(predict_key) + '\n')

                    # Get ks_templates results
                    filename = predict_versions_chroma_path + "ks_templates.txt"
                    os.makedirs(os.path.dirname(filename), exist_ok=True)
                    with open(filename, mode='w') as f:
                        for i in range(1, 25):
                            predict_key = predict_SWD_key(
                            get_SWD_Chroma(v, i, c),
                                ks_templates[0],
                                ks_templates[1],
                                pitches[v][c][i]
                            )
                            f.write(str(predict_key) + '\n')

                    # Get harmonic_templates results
                    filename = predict_versions_chroma_path + "harmonic_templates.txt"
                    os.makedirs(os.path.dirname(filename), exist_ok=True)
                    with open(filename, mode='w') as f:
                        for i in range(1, 25):
                            predict_key = predict_SWD_key(
                            get_SWD_Chroma(v, i, c),
                                harmonic_templates[0],
                                harmonic_templates[1],
                                pitches[v][c][i]
                            )
                            f.write(str(predict_key) + '\n')
            else:

                # Get binary_templates results
                filename = predict_versions_path + "binary_templates.txt"
                os.makedirs(os.path.dirname(filename), exist_ok=True)
                with open(filename, mode='w') as f:
                    for i in range(1, 25):
                        predict_key = predict_SWD_key(
                        get_SWD_Chroma(v, i),
                            binary_templates[0],
                            binary_templates[1],
                            pitches[v][i]
                        )
                        f.write(str(predict_key) + '\n')

                # Get ks_templates results
                filename = predict_versions_path + "ks_templates.txt"
                os.makedirs(os.path.dirname(filename), exist_ok=True)
                with open(filename, mode='w') as f:
                    for i in range(1, 25):
                        predict_key = predict_SWD_key(
                        get_SWD_Chroma(v, i),
                            ks_templates[0],
                            ks_templates[1],
                            pitches[v][i]
                        )
                        f.write(str(predict_key) + '\n')

                # Get harmonic_templates results
                filename = predict_versions_path + "harmonic_templates.txt"
                os.makedirs(os.path.dirname(filename), exist_ok=True)
                with open(filename, mode='w') as f:
                    for i in range(1, 25):
                        predict_key = predict_SWD_key(
                        get_SWD_Chroma(v, i),
                            harmonic_templates[0],
                            harmonic_templates[1],
                            pitches[v][i]
                        )
                        f.write(str(predict_key) + '\n')

In [19]:
create_SWD_predict(versions, chroma_type, pitches, binary_templates, ks_templates, harmonic_templates)

In [29]:
def get_raw_acc(p, y):
    
    count = 0
    for i in range(1, len(y)):
        if p[i] == y[i]:
            count += 1
    return round(count / (len(y)-1), 5)
    
def get_weighted_acc(p, y):
    """
    Refer to the weighted accuracy of Task 1 to see how weighting works.
    """
    
    score = 0
    for i in range(1, len(y)):
        if p[i] == y[i]:
            score += 1
        elif (p[i]+7)%12 == y[i]:
            score += 0.5
        elif p[i] < 12:
            if p[i] + 9 == y[i]:
                score += 0.3
            elif p[i] + 12 == y[i]:
                score += 0.2
        elif p[i] >= 12:
            if (p[i] + 3)%12 == y[i]:
                score += 0.3
            elif p[i] - 12 == y[i]:
                score += 0.2
    return round(score / (len(y)-1), 5)

def create_SWD_acc(versions:list, chroma_type:list):
    
    if(not os.path.isdir('result/SWD/acc/')):
        acc_path = "result/SWD/acc/"
        predict_path = "result/SWD/predict/"
        
        acc = {}
        for v in versions:
            
            if v == "midi":
                continue
            
            acc_versions_path = acc_path + v + ".txt"
            predict_versions_path = predict_path + "/" + v + "/"
            
            acc[v] = {}
            
            # Get ground truth
            y = keys[v]
            
            for c in chroma_type:
                acc_versions_chroma_path = acc_versions_path + "/" + c + "/"
                predict_versions_chroma_path = predict_versions_path + "/" + c + "/"
                acc[v][c] = {}
                
                # Get binary_templates acc
                acc[v][c]["binary_templates"] = {}
                p_bt_filename = predict_versions_chroma_path + "binary_templates.txt"
                p = [-1]
                with open(p_bt_filename, mode='r+') as f:
                    for i in range(1, 25):
                        p.append(int(f.readline().rstrip()))
                
                # Raw and weighted acc
                acc[v][c]["binary_templates"]["raw"] = get_raw_acc(p, y)
                acc[v][c]["binary_templates"]["weighted"] = get_weighted_acc(p, y)
                
                
                # Get ks_templates acc
                acc[v][c]["ks_templates"] = {}
                p_ks_filename = predict_versions_chroma_path + "ks_templates.txt"
                p = [-1]
                with open(p_ks_filename, mode='r+') as f:
                    for i in range(1, 25):
                        p.append(int(f.readline().rstrip()))
                
                # Raw and weighted acc
                acc[v][c]["ks_templates"]["raw"] = get_raw_acc(p, y)
                acc[v][c]["ks_templates"]["weighted"] = get_weighted_acc(p, y)
                
                
                # Get harmonic_templates acc
                acc[v][c]["harmonic_templates"] = {}
                p_hm_filename = predict_versions_chroma_path + "harmonic_templates.txt"
                p = [-1]
                with open(p_hm_filename, mode='r+') as f:
                    for i in range(1, 25):
                        p.append(int(f.readline().rstrip()))
                
                # Raw and weighted acc
                acc[v][c]["harmonic_templates"]["raw"] = get_raw_acc(p, y)
                acc[v][c]["harmonic_templates"]["weighted"] = get_weighted_acc(p, y)
        
        # MIDI
        v = "midi"
        acc[v] = {}
        predict_versions_path = predict_path + "/midi/"
        acc_versions_path = acc_path +"midi.txt"
        
        # Get binary_templates acc
        acc[v]["binary_templates"] = {}
        p_bt_filename = predict_versions_path + "binary_templates.txt"
        p = [-1]
        with open(p_bt_filename, mode='r+') as f:
            for i in range(1, 25):
                p.append(int(f.readline().rstrip()))

        # Raw and weighted acc
        acc[v]["binary_templates"]["raw"] = get_raw_acc(p, y)
        acc[v]["binary_templates"]["weighted"] = get_weighted_acc(p, y)


        # Get ks_templates acc
        acc[v]["ks_templates"] = {}
        p_ks_filename = predict_versions_path + "ks_templates.txt"
        p = [-1]
        with open(p_ks_filename, mode='r+') as f:
            for i in range(1, 25):
                p.append(int(f.readline().rstrip()))

        # Raw and weighted acc
        acc[v]["ks_templates"]["raw"] = get_raw_acc(p, y)
        acc[v]["ks_templates"]["weighted"] = get_weighted_acc(p, y)


        # Get harmonic_templates acc
        acc[v]["harmonic_templates"] = {}
        p_hm_filename = predict_versions_path + "harmonic_templates.txt"
        p = [-1]
        with open(p_hm_filename, mode='r+') as f:
            for i in range(1, 25):
                p.append(int(f.readline().rstrip()))

        # Raw and weighted acc
        acc[v]["harmonic_templates"]["raw"] = get_raw_acc(p, y)
        acc[v]["harmonic_templates"]["weighted"] = get_weighted_acc(p, y)
        
        
        return acc

In [30]:
SWD_ans = create_SWD_acc(versions, chroma_type)

In [31]:
SWD_ans
# The result looks suspicously round probably because of the lack of data count

{'HU33': {'stft': {'binary_templates': {'raw': 0.375, 'weighted': 0.375},
   'ks_templates': {'raw': 0.33333, 'weighted': 0.34167},
   'harmonic_templates': {'raw': 0.375, 'weighted': 0.375}},
  'cq': {'binary_templates': {'raw': 0.29167, 'weighted': 0.3},
   'ks_templates': {'raw': 0.33333, 'weighted': 0.33333},
   'harmonic_templates': {'raw': 0.29167, 'weighted': 0.3}},
  'cens': {'binary_templates': {'raw': 0.29167, 'weighted': 0.3},
   'ks_templates': {'raw': 0.33333, 'weighted': 0.33333},
   'harmonic_templates': {'raw': 0.29167, 'weighted': 0.3}}},
 'SC06': {'stft': {'binary_templates': {'raw': 0.33333, 'weighted': 0.33333},
   'ks_templates': {'raw': 0.33333, 'weighted': 0.33333},
   'harmonic_templates': {'raw': 0.33333, 'weighted': 0.33333}},
  'cq': {'binary_templates': {'raw': 0.33333, 'weighted': 0.33333},
   'ks_templates': {'raw': 0.33333, 'weighted': 0.33333},
   'harmonic_templates': {'raw': 0.33333, 'weighted': 0.33333}},
  'cens': {'binary_templates': {'raw': 0.29167